# `CLOUD.jl` - 2D linear advection example

In [9]:
using CLOUD, OrdinaryDiffEq, Plots, Plots.PlotMeasures

┌ Info: Precompiling CLOUD [fb992021-99c7-4c2d-a14b-5e48ac4045b2]
└ @ Base loading.jl:1423
ERROR: LoadError: InitError: Artifact "OpenSSL" was not installed correctly. Try `using Pkg; Pkg.instantiate()` to re-install all missing resources.
Stacktrace:
  [1] error(s::String)
    @ Base ./error.jl:33
  [2] _artifact_str(__module__::Module, artifacts_toml::String, name::SubString{String}, path_tail::String, artifact_dict::Dict{String, Any}, hash::Base.SHA1, platform::Base.BinaryPlatforms.Platform, lazyartifacts::Any)
    @ Artifacts /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Artifacts/src/Artifacts.jl:551
  [3] invokelatest(::Any, ::Any, ::Vararg{Any}; kwargs::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
    @ Base ./essentials.jl:716
  [4] invokelatest(::Any, ::Any, ::Vararg{Any})
    @ Base ./essentials.jl:714
  [5] macro expansion
    @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Artifacts/src/Artifact

    @ Base ./error.jl:33
  [2] compilecache(pkg::Base.PkgId, path::String, internal_stderr::IO, internal_stdout::IO, ignore_loaded_modules::Bool)
    @ Base ./loading.jl:1466
  [3] compilecache(pkg::Base.PkgId, path::String)
    @ Base ./loading.jl:1410
  [4] _require(pkg::Base.PkgId)
    @ Base ./loading.jl:1120
  [5] require(uuidkey::Base.PkgId)
    @ Base ./loading.jl:1013
  [6] require(into::Module, mod::Symbol)
    @ Base ./loading.jl:997
  [7] include(mod::Module, _path::String)
    @ Base ./Base.jl:418
  [8] include(x::String)
    @ CLOUD ~/Research/CLOUD.jl/src/CLOUD.jl:1
  [9] top-level scope
    @ ~/Research/CLOUD.jl/src/CLOUD.jl:15
 [10] include
    @ ./Base.jl:418 [inlined]
 [11] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::Nothing)
    @ Base ./loading.jl:1318
 [12] top-level scope
    @ none:1
 [13] eval
    @ ./boot.j

LoadError: Failed to precompile CLOUD [fb992021-99c7-4c2d-a14b-5e48ac4045b2] to /Users/tristanmontoya/.julia/compiled/v1.7/CLOUD/jl_4Zr9ip.

## Define the physical problem

In [10]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 10.0/(a*max(abs(cos(θ)),abs(sin(θ)))) # end time for one period
CFL = 0.025

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

LoadError: UndefVarError: LinearAdvectionEquation not defined

## Set discretization parameters (energy-conservative scheme)

In [11]:
M = 4
p = 4
form = WeakConservationForm(mapping_form=SkewSymmetricMapping(), 
    inviscid_numerical_flux=LaxFriedrichsNumericalFlux(0.0))
ode_algorithm = CarpenterKennedy2N54();

LoadError: UndefVarError: SkewSymmetricMapping not defined

## Set up a `NodalTensor` scheme on quadrilaterals of degree `p` with `M` edges in each direction

In [12]:
reference_approximation = ReferenceApproximation(
    NodalTensor(p), Tri(), mapping_degree=3)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M), collapsed_orientation=false), 
    reference_approximation.reference_element, ChanWarping(0.05,(1.0,1.0)))

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation, project_jacobian=true)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T),
    "results/advection_2d/", overwrite=true, clear=true)

plot(spatial_discretization, volume_quadrature=true, mapping_nodes=true,
    facet_quadrature=true, grid_connect=true,fmt = :png)

LoadError: UndefVarError: NodalTensor not defined

## Run the solver

In [13]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T))

h = L/sqrt(reference_approximation.N_p * spatial_discretization.N_e)
dt = CFL * h / a

CLOUD_reset_timer!()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, 
    callback=save_callback(results_path, (0.0,T), floor(Int, T/(dt*10))))
CLOUD_print_timer()

LoadError: UndefVarError: semidiscretize not defined

## Calculate the integral L<sup>2</sup> error norm

In [14]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)

LoadError: UndefVarError: ErrorAnalysis not defined

## Analyze conservation and energy dissipation

In [15]:
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
p1 = plot(analyze(conservation_analysis, load_time_steps(results_path)), ylabel="Conservation Functional")
p2 = plot(analyze(energy_analysis, load_time_steps(results_path)), ylabel="Energy Functional")
plot(p1,p2, size=(600,300),margin=5mm, fmt=:png)

LoadError: UndefVarError: PrimaryConservationAnalysis not defined

## Postprocess for VTK visualization

In [16]:
postprocess_vtk(spatial_discretization, string(results_path,"solution.vtu"), last(sol.u));

LoadError: UndefVarError: results_path not defined